In [1]:
# Filepath of dataset
dataset_fp = 'WatClaimCheck_dataset'

In [2]:
from helper import read_watclaimcheck_dataset
import pandas as pd

In [3]:
train_pd, valid_pd, test_pd = read_watclaimcheck_dataset(dataset_fp)

In [4]:
train_pd

,metadata,label
0,claimant,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
0,claim,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
0,claim_date,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
0,review_date,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
0,id,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
...,...,...
26975,claim,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
26975,claim_date,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
26975,review_date,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
26975,id,"{'reviewer_name': 'Alt News', 'reviewer_site':..."
